<a href="https://colab.research.google.com/github/anjana-psvel/Precog_Task/blob/main/fine_tuning_gpt3_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install -U openai
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 13.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 22.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 16.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.2/401.2 kB 38.5 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.20.3
    Uninstalling huggingface-hub-0.20.3:
      Successfully uninstalled huggingface-hub-0.20.3


In [4]:
from datasets import load_dataset

# Load a text dataset example
dataset = load_dataset("PiC/phrase_similarity")

# Accessing the dataset
train_data = dataset["train"]
test_data = dataset["test"]

# Print the first few examples
print(train_data[0])
print(test_data[0])


# prompt: create a dataframe with phrase1, phrase2, sentene1 and sentence2 with their label

import pandas as pd
data = {
    "phrase1": train_data["phrase1"],
    "phrase2": train_data["phrase2"],
    "sentence1": train_data["sentence1"],
    "sentence2": train_data["sentence2"],
    "label": train_data["label"]
}

df = pd.DataFrame(data)
df.head()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/7004 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/2000 [00:00<?, ? examples/s]

{'phrase1': 'newly formed camp', 'phrase2': 'recently made encampment', 'sentence1': 'newly formed camp is released from the membrane and diffuses across the intracellular space where it serves to activate pka.', 'sentence2': 'recently made encampment is released from the membrane and diffuses across the intracellular space where it serves to activate pka.', 'label': 0, 'idx': 0}
{'phrase1': 'air position', 'phrase2': 'posture while jumping', 'sentence1': 'In 1990, Petit accepted a full-time overnight on air position at gospel radio station WYLD-AM.', 'sentence2': 'In 1990, Petit accepted a full-time overnight on posture while jumping at gospel radio station WYLD-AM.', 'label': 0, 'idx': 0}


,phrase1,phrase2,sentence1,sentence2,label
0,newly formed camp,recently made encampment,newly formed camp is released from the membran...,recently made encampment is released from the ...,0
1,one data,a particular statistic,"According to one data, in 1910, on others – in...","According to a particular statistic, in 1910, ...",1
2,particular structure,specific edifice,Note that Fact 1 does not assume any particula...,Note that Fact 1 does not assume any specific ...,0
3,involved people,participating individuals,Assessment-Center are usually group-processes ...,Assessment-Center are usually group-processes ...,1
4,different cross,opposing inquiries,"At the end of the 1980s, a different cross had...","At the end of the 1980s, a opposing inquiries ...",0


In [7]:
#format dataset
import json

def convert_to_gpt35_format(dataset):
    fine_tuning_data = []
    for _, row in dataset.iterrows():
        # json_response = '{"Label": "' + row['label'] + '"}'
        label_value = int(row['label'])

        # Create a dictionary with the label
        json_response = label_value

        # Convert the dictionary to a JSON string
        json_string = json.dumps(json_response)


        fine_tuning_data.append({
            "messages": [
                {"phrase1": row['phrase1'], "phrase2": row['phrase2']},
                {"label": json_response}
            ]
        })
    return fine_tuning_data

# dataset = pd.read_csv('/content/back_support_train.csv')
converted_data = convert_to_gpt35_format(df)
converted_data[0]['messages']

[{'phrase1': 'newly formed camp', 'phrase2': 'recently made encampment'},
 {'label': 0}]

In [11]:
from sklearn.model_selection import train_test_split

# Stratified splitting. Assuming 'Top Category' can be used for stratification
train_data, val_data = train_test_split(
    converted_data,
    test_size=0.2,
    stratify=df['label'],
    random_state=42  # for reproducibility
)



In [12]:
def write_to_jsonl(data, file_path):
    with open(file_path, 'w') as file:
        for entry in data:
            json.dump(entry, file)
            file.write('\n')


training_file_name = "train.jsonl"
validation_file_name = "val.jsonl"

write_to_jsonl(train_data, training_file_name)
write_to_jsonl(val_data, validation_file_name)

In [13]:
from openai import OpenAI
client = OpenAI(api_key="sk-proj-vyrdktoUURQQ5yjSOB4ZT3BlbkFJg8oth2JvAdIqFk1LfBNm")

In [14]:

training_file = client.files.create(
    file=open(training_file_name, "rb"), purpose="fine-tune"
)
validation_file = client.files.create(
    file=open(validation_file_name, "rb"), purpose="fine-tune"
)

print("Training file id:", training_file.id)
print("Validation file id:", validation_file.id)

Training file id: file-kKl8js6dO1ovaDUUgqXIljGS
Validation file id: file-J7QrDHMLyt3WxLSoIEUSgl64


In [16]:
suffix_name = "phrase_similarity"

response = client.fine_tuning.jobs.create(
    training_file=training_file.id,
    validation_file=validation_file.id,
    model="gpt-3.5-turbo",
    suffix=suffix_name,
)
response


BadRequestError: Error code: 400 - {'error': {'message': 'Fine-tuning jobs cannot be created on an Explore plan. You can upgrade to a paid plan on your billing page: https://platform.openai.com/account/billing/overview', 'type': 'invalid_request_error', 'param': None, 'code': 'exceeded_quota'}}